In [1]:
import os
import cv2
import numpy as np
import shutil
import random
import argparse

In [4]:
images_num=100
image_size=416
images_path="./Images/"
labels_txt="./labels.txt"
small=6
medium=12
big=0

SIZE = image_size

if os.path.exists(images_path): shutil.rmtree(images_path)
os.mkdir(images_path)

image_paths  = [os.path.join(os.path.realpath("."), "./mahjong/train/" + image_name) for image_name in os.listdir("./mahjong/train")]
image_paths += [os.path.join(os.path.realpath("."), "./mahjong/test/"  + image_name) for image_name in os.listdir("./mahjong/test")]

In [5]:
def compute_iou(box1, box2):
    """xmin, ymin, xmax, ymax"""

    A1 = (box1[2] - box1[0])*(box1[3] - box1[1])
    A2 = (box2[2] - box2[0])*(box2[3] - box2[1])

    xmin = max(box1[0], box2[0])
    ymin = max(box1[1], box2[1])
    xmax = min(box1[2], box2[2])
    ymax = min(box1[3], box2[3])

    if ymin >= ymax or xmin >= xmax: return 0
    return  ((xmax-xmin) * (ymax - ymin)) / (A1 + A2)

In [6]:
def make_image(data, image_path, ratio=1):

    blank = data[0]
    boxes = data[1]
    label = data[2]

    ID = image_path.split("/")[-1].split('_')[0]
    image = cv2.imread(image_path)
    image = cv2.resize(image, (int(24*ratio), int(32*ratio)))
    h, w, c = image.shape

    while True:
        xmin = np.random.randint(0, SIZE-w, 1)[0]
        ymin = np.random.randint(0, SIZE-h, 1)[0]
        xmax = xmin + w
        ymax = ymin + h
        box = [xmin, ymin, xmax, ymax]

        iou = [compute_iou(box, b) for b in boxes]
        if max(iou) < 0.02:
            boxes.append(box)
            label.append(ID)
            break

    for i in range(w):
        for j in range(h):
            x = xmin + i
            y = ymin + j
            blank[y][x] = image[j][i]

    # cv2.rectangle(blank, (xmin, ymin), (xmax, ymax), [0, 0, 255], 2)
    return blank

In [7]:
with open(labels_txt, "w") as wf:
    image_num1 = 0
    while image_num1 < images_num:
        image_path = os.path.realpath(os.path.join(images_path, "%06d.jpg" %(image_num1+1)))
        annotation = image_path
        blanks = np.ones(shape=[SIZE, SIZE, 3]) * 255
        bboxes = [[0,0,1,1]]
        labels = [0]
        data = [blanks, bboxes, labels]
        bboxes_num = 0

        # small object
        ratios = [0.5, 0.8]
        N = random.randint(0, small)
        if N !=0: bboxes_num += 1
        for _ in range(N):
            ratio = random.choice(ratios)
            idx = random.randint(0, 600)
            data[0] = make_image(data, image_paths[idx], ratio)

        # medium object
        ratios = [1., 1.5, 2.]
        N = random.randint(0, medium)
        if N !=0: bboxes_num += 1
        for _ in range(N):
            ratio = random.choice(ratios)
            idx = random.randint(0, 600)
            data[0] = make_image(data, image_paths[idx], ratio)

        # big object
        ratios = [3., 4.]
        N = random.randint(0, big)
        if N !=0: bboxes_num += 1
        for _ in range(N):
            ratio = random.choice(ratios)
            idx = random.randint(0, 600)
            data[0] = make_image(data, image_paths[idx], ratio)

        if bboxes_num == 0: continue
        cv2.imwrite(image_path, data[0])
        for i in range(len(labels)):
            if i == 0: continue
            xmin = str(bboxes[i][0])
            ymin = str(bboxes[i][1])
            xmax = str(bboxes[i][2])
            ymax = str(bboxes[i][3])
            class_ind = str(labels[i])
            annotation += ' ' + ','.join([xmin, ymin, xmax, ymax, str(class_ind)])
        image_num1 += 1
        print("=> %s" %annotation)
        wf.write(annotation + "\n")

=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000001.jpg 255,77,274,102,36 107,196,119,212,13 155,197,174,222,34 258,10,294,58,29 371,149,395,181,17 13,27,37,59,5 250,218,286,266,3 290,117,314,149,42 138,65,186,129,26 30,348,78,412,27 86,9,110,41,11 49,112,97,176,14 41,2,65,34,18
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000002.jpg 100,115,148,179,30 237,225,285,289,33
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000003.jpg 233,20,245,36,32 49,187,68,212,36 269,339,293,371,42 312,328,360,392,28 328,154,376,218,36 327,225,363,273,17 55,123,103,187,28 223,96,247,128,11 281,247,329,311,26 112,115,136,147,32 212,45,248,93,2 28,0,52,32,27 376,187,400,219,18
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000004.jpg 66,254,78,270,7 107,165,155,229,2 158,29,182,61,2

=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000038.jpg 61,10,73,26,5 251,4,275,36,22 140,133,188,197,6 143,337,191,401,41 261,174,309,238,33 158,209,194,257,10 9,113,45,161,27 82,328,106,360,13
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000039.jpg 288,18,300,34,12 256,108,275,133,7 139,172,158,197,25 262,328,281,353,5 99,207,123,239,21 286,146,310,178,2
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000040.jpg 192,228,211,253,17 356,364,375,389,16 33,286,69,334,30 204,257,252,321,34 270,363,306,411,11 185,44,221,92,7 351,249,387,297,28 225,320,261,368,19 298,275,346,339,19
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000041.jpg 254,43,266,59,36 366,321,378,337,6 266,183,278,199,11 401,276,413,292,42 364,232,376,248,37 137,9,173,57,10 17,4,53,52,39
=> D:\Use

=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000075.jpg 42,361,61,386,41 236,304,255,329,7 124,52,136,68,27 198,50,210,66,23 267,244,279,260,20 153,221,201,285,38 365,308,401,356,28 338,0,386,64,16 147,70,183,118,32 22,265,46,297,11
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000076.jpg 178,352,197,377,34 256,49,280,81,1 258,195,294,243,33
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000077.jpg 313,209,332,234,28 358,263,377,288,29 393,371,412,396,10 183,288,195,304,37 317,250,329,266,19 225,291,261,339,17 216,107,252,155,39 159,233,195,281,6 83,145,119,193,1 221,9,269,73,35 88,90,112,122,19
=> D:\Users\charl\Desktop\mysql\TensorFlow2.0-Examples-master\4-Object_Detection\YOLOV3\yymnist\Images\000078.jpg 66,12,85,37,32 358,159,394,207,40 249,181,273,213,14 28,137,52,169,28 117,305,141,337,6 101,344,149,408,22 234,3